In [1]:
from winners_only import *
from log import *
from historical import *
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("data/updated_data.csv")

In [3]:
X_train, y_train, test_2019 = historical_cleaning(df)

In [4]:
n_estimators = [100,250,500,750,1000,1500,2000,2500]
max_depth = [2,4,6,8]
learning_rate = [0.1,0.3,0.5]
subsample = [0.25,0.5,0.75]



In [5]:
# n_estimators = [100,250]
# max_depth = [2]
# learning_rate = [0.1]
# subsample = [0.25]


In [6]:
param_info = []
acc_info = []

In [8]:
for n in n_estimators:
    for d in max_depth:
        for lr in learning_rate:
            for sub in subsample:
                XGBR = xgb.XGBClassifier(colsample_bytree=0.4, gamma=0, learning_rate=lr, max_depth=d, 
                min_child_weight=1.5, n_estimators=n, reg_alpha=0.75, reg_lambda=0.45, subsample=sub, seed=42)
                #
                XGBR.fit(X_train,y_train)
                param_info.append([n,d,lr,sub])
                prediction = pd.DataFrame(test_2019["Fighter"])
                test_2019_clean = test_2019.drop(columns = ["Fighter","Result"])
                prediction["Win Probability"] = XGBR.predict_proba(test_2019_clean)[:,1]
                prediction = prediction.set_index("Fighter")
                p2019 = prediction.to_dict()["Win Probability"]
                fighter_data = list(p2019.keys())
                check = df
                check["year"] = check["date"].apply(lambda x: x[:-6])
                check = check[check["year"]=="2019"]
                check = check[~check["B_avg_BODY_att"].isnull()]
                check = check[check["R_fighter"].apply(lambda x: x in fighter_data)]
                check = check[check["B_fighter"].apply(lambda x: x in fighter_data)]
                comparison = check[["R_fighter","B_fighter","Winner"]]
                comparison["R_Fighter_Proba"] = comparison["R_fighter"].apply(lambda x: p2019[x] if x in list(p2019.keys()) else 0)
                comparison["B_Fighter_Proba"] = comparison["B_fighter"].apply(lambda x: p2019[x] if x in list(p2019.keys()) else 0)
                comparison["Proba Results"] = comparison.apply(lambda x: "Red" if x["R_Fighter_Proba"] >= x["B_Fighter_Proba"] else "Blue",axis=1)
                comparison["Correct"] = comparison["Winner"] == comparison["Proba Results"]
                a = list(comparison["Correct"])
                acc = np.mean(a)
                acc_info.append(acc)
                print([n,d,lr,sub],acc)

[100, 2, 0.1, 0.25] 0.5773809523809523
[100, 2, 0.1, 0.5] 0.6011904761904762
[100, 2, 0.1, 0.75] 0.6428571428571429
[100, 2, 0.3, 0.25] 0.5892857142857143
[100, 2, 0.3, 0.5] 0.5416666666666666
[100, 2, 0.3, 0.75] 0.6011904761904762
[100, 2, 0.5, 0.25] 0.5535714285714286
[100, 2, 0.5, 0.5] 0.5357142857142857
[100, 2, 0.5, 0.75] 0.5714285714285714
[100, 4, 0.1, 0.25] 0.5238095238095238
[100, 4, 0.1, 0.5] 0.5416666666666666
[100, 4, 0.1, 0.75] 0.6130952380952381
[100, 4, 0.3, 0.25] 0.5178571428571429
[100, 4, 0.3, 0.5] 0.5297619047619048
[100, 4, 0.3, 0.75] 0.49404761904761907
[100, 4, 0.5, 0.25] 0.5059523809523809
[100, 4, 0.5, 0.5] 0.5476190476190477
[100, 4, 0.5, 0.75] 0.5297619047619048
[100, 6, 0.1, 0.25] 0.5714285714285714
[100, 6, 0.1, 0.5] 0.5773809523809523
[100, 6, 0.1, 0.75] 0.5952380952380952
[100, 6, 0.3, 0.25] 0.5714285714285714
[100, 6, 0.3, 0.5] 0.5357142857142857
[100, 6, 0.3, 0.75] 0.4880952380952381
[100, 6, 0.5, 0.25] 0.5476190476190477
[100, 6, 0.5, 0.5] 0.47023809523

[1500, 8, 0.5, 0.75] 0.5238095238095238
[2000, 2, 0.1, 0.25] 0.5833333333333334
[2000, 2, 0.1, 0.5] 0.5952380952380952
[2000, 2, 0.1, 0.75] 0.5535714285714286
[2000, 2, 0.3, 0.25] 0.5892857142857143
[2000, 2, 0.3, 0.5] 0.5416666666666666
[2000, 2, 0.3, 0.75] 0.48214285714285715
[2000, 2, 0.5, 0.25] 0.5
[2000, 2, 0.5, 0.5] 0.47619047619047616
[2000, 2, 0.5, 0.75] 0.5416666666666666
[2000, 4, 0.1, 0.25] 0.5595238095238095
[2000, 4, 0.1, 0.5] 0.5654761904761905
[2000, 4, 0.1, 0.75] 0.5059523809523809
[2000, 4, 0.3, 0.25] 0.5119047619047619
[2000, 4, 0.3, 0.5] 0.5119047619047619
[2000, 4, 0.3, 0.75] 0.5119047619047619
[2000, 4, 0.5, 0.25] 0.5238095238095238
[2000, 4, 0.5, 0.5] 0.5773809523809523
[2000, 4, 0.5, 0.75] 0.5059523809523809
[2000, 6, 0.1, 0.25] 0.5178571428571429
[2000, 6, 0.1, 0.5] 0.5059523809523809
[2000, 6, 0.1, 0.75] 0.4583333333333333
[2000, 6, 0.3, 0.25] 0.5535714285714286
[2000, 6, 0.3, 0.5] 0.5059523809523809
[2000, 6, 0.3, 0.75] 0.5297619047619048
[2000, 6, 0.5, 0.25] 

In [19]:

grid_results = sorted(zip(acc_info,param_info),reverse=True)
frame = pd.DataFrame(grid_results)

In [22]:
frame.to_csv("grid_results.csv",index=False)

In [15]:
frame.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288
0,"[100, 2, 0.1, 0.25]","[100, 2, 0.1, 0.25]","[100, 2, 0.1, 0.5]","[100, 2, 0.1, 0.75]","[100, 2, 0.3, 0.25]","[100, 2, 0.3, 0.5]","[100, 2, 0.3, 0.75]","[100, 2, 0.5, 0.25]","[100, 2, 0.5, 0.5]","[100, 2, 0.5, 0.75]","[100, 4, 0.1, 0.25]","[100, 4, 0.1, 0.5]","[100, 4, 0.1, 0.75]","[100, 4, 0.3, 0.25]","[100, 4, 0.3, 0.5]","[100, 4, 0.3, 0.75]","[100, 4, 0.5, 0.25]","[100, 4, 0.5, 0.5]","[100, 4, 0.5, 0.75]","[100, 6, 0.1, 0.25]","[100, 6, 0.1, 0.5]","[100, 6, 0.1, 0.75]","[100, 6, 0.3, 0.25]","[100, 6, 0.3, 0.5]","[100, 6, 0.3, 0.75]","[100, 6, 0.5, 0.25]","[100, 6, 0.5, 0.5]","[100, 6, 0.5, 0.75]","[100, 8, 0.1, 0.25]","[100, 8, 0.1, 0.5]","[100, 8, 0.1, 0.75]","[100, 8, 0.3, 0.25]","[100, 8, 0.3, 0.5]","[100, 8, 0.3, 0.75]","[100, 8, 0.5, 0.25]","[100, 8, 0.5, 0.5]","[100, 8, 0.5, 0.75]","[250, 2, 0.1, 0.25]","[250, 2, 0.1, 0.5]","[250, 2, 0.1, 0.75]","[250, 2, 0.3, 0.25]","[250, 2, 0.3, 0.5]","[250, 2, 0.3, 0.75]","[250, 2, 0.5, 0.25]","[250, 2, 0.5, 0.5]","[250, 2, 0.5, 0.75]","[250, 4, 0.1, 0.25]","[250, 4, 0.1, 0.5]","[250, 4, 0.1, 0.75]","[250, 4, 0.3, 0.25]","[250, 4, 0.3, 0.5]","[250, 4, 0.3, 0.75]","[250, 4, 0.5, 0.25]","[250, 4, 0.5, 0.5]","[250, 4, 0.5, 0.75]","[250, 6, 0.1, 0.25]","[250, 6, 0.1, 0.5]","[250, 6, 0.1, 0.75]","[250, 6, 0.3, 0.25]","[250, 6, 0.3, 0.5]","[250, 6, 0.3, 0.75]","[250, 6, 0.5, 0.25]","[250, 6, 0.5, 0.5]","[250, 6, 0.5, 0.75]","[250, 8, 0.1, 0.25]","[250, 8, 0.1, 0.5]","[250, 8, 0.1, 0.75]","[250, 8, 0.3, 0.25]","[250, 8, 0.3, 0.5]","[250, 8, 0.3, 0.75]","[250, 8, 0.5, 0.25]","[250, 8, 0.5, 0.5]","[250, 8, 0.5, 0.75]","[500, 2, 0.1, 0.25]","[500, 2, 0.1, 0.5]","[500, 2, 0.1, 0.75]","[500, 2, 0.3, 0.25]","[500, 2, 0.3, 0.5]","[500, 2, 0.3, 0.75]","[500, 2, 0.5, 0.25]","[500, 2, 0.5, 0.5]","[500, 2, 0.5, 0.75]","[500, 4, 0.1, 0.25]","[500, 4, 0.1, 0.5]","[500, 4, 0.1, 0.75]","[500, 4, 0.3, 0.25]","[500, 4, 0.3, 0.5]","[500, 4, 0.3, 0.75]","[500, 4, 0.5, 0.25]","[500, 4, 0.5, 0.5]","[500, 4, 0.5, 0.75]","[500, 6, 0.1, 0.25]","[500, 6, 0.1, 0.5]","[500, 6, 0.1, 0.75]","[500, 6, 0.3, 0.25]","[500, 6, 0.3, 0.5]","[500, 6, 0.3, 0.75]","[500, 6, 0.5, 0.25]","[500, 6, 0.5, 0.5]","[500, 6, 0.5, 0.75]","[500, 8, 0.1, 0.25]","[500, 8, 0.1, 0.5]","[500, 8, 0.1, 0.75]","[500, 8, 0.3, 0.25]","[500, 8, 0.3, 0.5]","[500, 8, 0.3, 0.75]","[500, 8, 0.5, 0.25]","[500, 8, 0.5, 0.5]","[500, 8, 0.5, 0.75]","[750, 2, 0.1, 0.25]","[750, 2, 0.1, 0.5]","[750, 2, 0.1, 0.75]","[750, 2, 0.3, 0.25]","[750, 2, 0.3, 0.5]","[750, 2, 0.3, 0.75]","[750, 2, 0.5, 0.25]","[750, 2, 0.5, 0.5]","[750, 2, 0.5, 0.75]","[750, 4, 0.1, 0.25]","[750, 4, 0.1, 0.5]","[750, 4, 0.1, 0.75]","[750, 4, 0.3, 0.25]","[750, 4, 0.3, 0.5]","[750, 4, 0.3, 0.75]","[750, 4, 0.5, 0.25]","[750, 4, 0.5, 0.5]","[750, 4, 0.5, 0.75]","[750, 6, 0.1, 0.25]","[750, 6, 0.1, 0.5]","[750, 6, 0.1, 0.75]","[750, 6, 0.3, 0.25]","[750, 6, 0.3, 0.5]","[750, 6, 0.3, 0.75]","[750, 6, 0.5, 0.25]","[750, 6, 0.5, 0.5]","[750, 6, 0.5, 0.75]","[75

# TRAIN XG BOOST MODEL

In [4]:
XGBR = xgb.XGBClassifier(colsample_bytree=0.4,
                gamma=0,
                learning_rate=0.07,
                max_depth=3,
                min_child_weight=1.5,
                n_estimators=750,
                reg_alpha=0.75,
                reg_lambda=0.45,
                subsample=0.6,
                seed=42)

XGBR.fit(X_train,y_train)



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, gamma=0, learning_rate=0.07, max_delta_step=0,
       max_depth=3, min_child_weight=1.5, missing=None, n_estimators=750,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
       silent=True, subsample=0.6)

# Predict 2019 Fights

In [43]:
#Prediction numbers on cleaned 2019 Fighter Data

prediction = pd.DataFrame(test_2019["Fighter"])
test_2019 = test_2019.drop(columns = ["Fighter","Result"])
prediction["Win Probability"] = XGBR.predict_proba(test_2019)[:,1]
prediction = prediction.set_index("Fighter")
p2019 = prediction.to_dict()["Win Probability"]


In [47]:

check = df
check["year"] = check["date"].apply(lambda x: x[:-6])
check = check[check["year"]=="2019"]
check = check[~check["B_avg_BODY_att"].isnull()]
check = check[check["R_fighter"].apply(lambda x: x in fighter_data)]
check = check[check["B_fighter"].apply(lambda x: x in fighter_data)]



comparison = check[["R_fighter","B_fighter","Winner"]]
comparison["R_Fighter_Proba"] = comparison["R_fighter"].apply(lambda x: p2019[x] if x in list(p2019.keys()) else 0)
comparison["B_Fighter_Proba"] = comparison["B_fighter"].apply(lambda x: p2019[x] if x in list(p2019.keys()) else 0)


comparison["Proba Results"] = comparison.apply(lambda x: "Red" if x["R_Fighter_Proba"] >= x["B_Fighter_Proba"] else "Blue",axis=1)
comparison["Correct"] = comparison["Winner"] == comparison["Proba Results"]
a = list(comparison["Correct"])
acc = np.mean(a)

comparison["P_D"] = comparison["R_Fighter_Proba"] - comparison["B_Fighter_Proba"]
comparison["Pos_cumsum"] = np.sum(comparison["Correct"]*np.abs(comparison["P_D"]))
comparison["Neg_cumsum"] = np.sum((~comparison["Correct"])*np.abs(comparison["P_D"]))
comparison.describe()

,R_Fighter_Proba,B_Fighter_Proba,P_D,Pos_cumsum,Neg_cumsum
count,168.000000,168.000000,168.000000,168.000000,168.0000
mean,0.498040,0.492130,0.005910,15.980126,9.2969
std,0.149753,0.141873,0.187230,0.000000,0.0000
min,0.092437,0.132982,-0.438630,15.980126,9.2969
25%,0.409664,0.393496,-0.110677,15.980126,9.2969
50%,0.479402,0.478297,0.004243,15.980126,9.2969
75%,0.600693,0.597139,0.127150,15.980126,9.2969
max,0.885466,0.818711,0.542639,15.980126,9.2969


In [48]:
comparison

,R_fighter,B_fighter,Winner,R_Fighter_Proba,B_Fighter_Proba,Proba Results,Correct,P_D,Pos_cumsum,Neg_cumsum
0,Henry Cejudo,Marlon Moraes,Red,0.532230,0.401620,Red,True,0.130610,15.980126,9.2969
1,Valentina Shevchenko,Jessica Eye,Red,0.694044,0.508747,Red,True,0.185297,15.980126,9.2969
2,Tony Ferguson,Donald Cerrone,Red,0.677833,0.459667,Red,True,0.218166,15.980126,9.2969
3,Jimmie Rivera,Petr Yan,Blue,0.668925,0.596413,Red,False,0.072512,15.980126,9.2969
4,Tai Tuivasa,Blagoy Ivanov,Blue,0.511598,0.309827,Red,False,0.201772,15.980126,9.2969
5,Tatiana Suarez,Nina Ansaroff,Red,0.857667,0.533941,Red,True,0.323726,15.980126,9.2969
6,Aljamain Sterling,Pedro Munhoz,Red,0.705832,0.746159,Blue,False,-0.040327,15.980126,9.2969
7,Karolina Kowalkiewicz,Alexa Grasso,Blue,0.416342,0.522301,Blue,True,-0.105958,15.980126,9.2969
8,Ricardo Lamas,Calvin Kattar,Blue,0.410589,0.546407,Blue,True,-0.135817,15.980126,9.2969
9,Yan Xiaonan,Angela Hill,Red,0.572691,0.316535,Red,True,0.256156,15.980126,9.2969


In [46]:
acc

0.625

In [49]:
comparison["Winner"].value_counts()

Blue    86
Red     82
Name: Winner, dtype: int64

In [32]:
check.head(168)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_total_rounds_fought,B_total_time_fought(seconds),B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_BODY_att,R_avg_BODY_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_KD,R_avg_LEG_att,R_avg_LEG_landed,R_avg_PASS,R_avg_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_longest_win_streak,R_losses,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,year
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019-06-08,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0.0,4.0,0.0,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.600000,5.400000,0.400000,0.000000,65.400000,22.600000,0.466000,0.400000,0.800000,0.200000,0.100000,66.400000,23.600000,4.0,1.0,6.400000,4.000000,1.000000,0.600000,51.200000,17.400000,0.600000,0.200000,39.600000,9.400000,0.200000,6.800000,4.800000,0.000000,0.000000,52.800000,18.200000,0.236000,0.000000,1.000000,0.400000,0.100000,53.800000,19.200000,9.0,419.400000,0.0,0.0,1.0,0.0,2.0,1.0,0.0,4.0,Orthodox,167.64,170.18,135.0,0.0,4.0,0.0,21.900000,16.400000,17.000000,11.000000,75.000000,26.500000,9.400000,6.500000,74.200000,23.900000,0.400000,5.300000,3.700000,1.200000,0.000000,101.400000,44.000000,0.466000,0.100000,5.300000,1.900000,0.458000,129.900000,69.100000,4.0,2.0,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27.0,742.600000,3.0,0.0,2.0,4.0,2.0,0.0,0.0,8.0,Orthodox,162.56,162.56,135.0,31.0,32.0,2019
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019-06-08,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0.0,3.0,0.0,14.600000,9.100000,11.800000,7.300000,124.700000,42.100000,2.400000,1.900

In [19]:
rf = list(check["R_fighter"])

In [20]:
bf = list(check["B_fighter"])

In [24]:
fighter_data = list(p2019.keys())

In [25]:
for each in rf:
    print(each in fighter_data)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
False
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
